In [ ]:
!pip install 'bs4==0.0.1' 'lxml==4.9.2' 'googlemaps==4.10.0'

In [23]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import json
import time
import os
import googlemaps

# API key for places, directions, geocode and another to maps js
gmaps = googlemaps.Client(key=os.environ.get("GOOGLE_KEY"))

locations = []
geo = []
names = []

def get_google(query, retry=3):
    try:
        return gmaps.geocode(query)
    except Exception as e:
        if retry:
            time.sleep(1)
            return get_google(query, retry-1)
        else:
            print(f"query error {query}")
            print(e)
            return False

for i in range(1, 80):
    req = Request(f"http://www.comidadibuteco.com.br/category/butecos/page/{i}/")
    html_page = urlopen(req)

    soup = BeautifulSoup(html_page, "lxml")

    inner = soup.findAll('div', {"class": "result-inner"})
    for div in inner:
        for link in div.findAll('a'):
            href = link.get('href')
            if href.startswith('https://www.google.com/maps/search/?api='):
                geocode_result = get_google(href.split("query=")[1])
                if geocode_result:
                    geo.append(geocode_result[0]["geometry"]["location"])
                else:
                    geo.append({})
                break
        names.append(div.find('h2').text)
    print(f"foi página {i}")
    time.sleep(10)
for i in range(len(names)):
    locations.append({
        "name": names[i],
        "geo": geo[i]
    })

In [28]:
with open("places.json", "w") as f:
    f.write(json.dumps(locations, ensure_ascii=False))

In [24]:
len(names)

941

In [25]:
len(geo)

941